# Check Events

Check event rankings from GP3S.

Copyright 2022 Michael George (AKA Logiqx).

This file is part of GP3S Query and is distributed under the terms of the GNU General Public License.

GP3S Query is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GP3S Query is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GP3S Query. If not, see https://www.gnu.org/licenses/.

## Import Common Modules

In [1]:
import os
import sys
from datetime import datetime

import urllib.parse

import json

## Lookups

Obtained from gps-speedsurfing.com

In [2]:
users = {}

SITES = {
    'gps-speed': 'https://www.gps-speedsurfing.com/default.aspx?mnu=events',
    'gps-foil': 'https://www.gps-foilsurfing.com/default.aspx?mnu=events',
    'gps-wing': 'https://www.gps-wingfoiling.com/default.aspx?mnu=events',
    'gps-kite': 'https://www.gps-kitesurfing.com/default.aspx?mnu=events',
    'gps-ice': 'https://www.gps-icesailing.com/default.aspx?mnu=events'
}

speedTypes = {
    "speed_avg": "Average speed",
    "speed_100": "100 m run",
    "speed_250": "250 m run",
    "speed_500": "500 m run",
    "speed_halfhour": "1/2 hour speed",
    "speed_hour": "1 hour avg speed",
    "speed_24hour": "24 hour",
    "speed_mile": "Nautical mile",
    "speed_10sec": "Max. 10 sec. run",
    "speed_2sec": "Max. 2 sec.",
    "speed_formula": "Formula",
    "speed_alpha_racing": "Alpha racing",
    "speed_foil": "Foil"
}

## Load Events

In [3]:
def getUid(sessionUrl):
    """Get user ID from session URL"""

    parsedUrl = urllib.parse.urlparse(sessionUrl)
    uid = urllib.parse.parse_qs(parsedUrl.query)["uid"][0]
            
    return int(uid)

In [4]:
def loadEventRankings(eventDir):
    """Load event rankings"""
    
    rankings = {}

    if os.path.exists(eventDir):
        for rankingsFile in sorted(os.listdir(eventDir)):
            filename = os.path.join(eventDir, rankingsFile)
            with open(filename) as f:
                speedType = os.path.splitext(rankingsFile)[0]
                rankings[speedType] = json.load(f)

    return rankings

In [5]:
def parseEventRankings(rankings):
    """Parse event rankings"""

    bests = {}
    duplicates = {}
    
    for speedType, rows in rankings.items():
        if speedType not in bests:
            bests[speedType] = {}
           
        for row in rows:
            uid = getUid(row['sessionurl'])
            
            if uid not in users:
                users[uid] = row['username']

            if uid not in bests[speedType]:
                bests[speedType][uid] = row['speed']
            else:
                if speedType not in duplicates:
                    duplicates[speedType] = {}
                duplicates[speedType][uid] = bests[speedType][uid]

    return bests, duplicates

In [6]:
def loadEventSessions(apiName, eventId):
    """Load event sessions"""
    
    sessions = {}

    filename = os.path.join(projdir, 'cache', apiName, 'eventsessions', str(eventId) + '.json')
    with open(filename) as f:
        try:
            sessions = json.load(f)
        except Exception:
            print('Issue parsing {}'.format(filename))
            raise

    return sessions

In [7]:
def parseEventSessions(sessions):
    """Parse event sessions"""

    bests = {}
    dateIssues = 0
    
    for session in sessions:
        eventId = session['event_id']

        uid = getUid(session['sessionurl'])

        if uid not in users:
            users[uid] = session['username']

        # Check that session is really during the event
        #sessionDate = datetime.strptime(session['session_date'][:10], '%Y-%m-%d').strftime('%Y-%m-%d')
        sessionDate = datetime.strptime(session['session_date'].split(' ')[0], '%m/%d/%Y').strftime('%Y-%m-%d')
        if events[eventId]['start_date'][:10] <= sessionDate <= events[eventId]['end_date'][:10]:
            for speedType in session:
                if speedType.startswith('speed_'):
                    if speedType not in bests:
                        bests[speedType] = {}

                    if uid not in bests[speedType]:
                        bests[speedType][uid] = 0

                    if session[speedType] > bests[speedType][uid]:
                        bests[speedType][uid] = session[speedType]
            
        else:
            dateIssues += 1

    return bests, dateIssues

## Compare Bests

Compare best speeds in sessions and rankings

In [8]:
def speedKnots(speed):
    '''Calculate speed in knots'''

    return round(speed / 1.852, 2)

    
def addToRebuilds(rebuilds, uid, speedType):
    '''Add user to rebuild list'''

    if uid not in rebuilds:
        rebuilds[uid] = []
    if speedType not in rebuilds[uid]:
        rebuilds[uid].append(speedType)


def reportDuplicates(f, eventId, rankingDuplicates, rebuilds):
    '''Report duplicate usernames'''

    count = 0

    for speedType, uids in rankingDuplicates.items():
        for uid, speed in uids.items():
            if count == 0:
                f.write('### Duplicates\n\n')
                f.write('These rankings are duplicated.\n\n')

                f.write('| Speed Type | User Name | User ID | Speed (knots) |\n')
                f.write('| ---------- | --------- | :-----: | :-----------: |\n')

            f.write('| {} | {} | {} | {:.2f} |\n'.format(speedTypes[speedType], users[uid], uid, speedKnots(speed)))

            addToRebuilds(rebuilds, uid, speedType)

            count += 1

    if count:
        f.write('\n')

    return count
               

def reportGhosts(f, eventId, rankingBests, sessionBests, rebuilds):
    '''Report ghost rankings'''

    count = 0

    for speedType, uids in rankingBests.items():
        if speedType not in sessionBests:
            print('Warning: {} not in event sessions of {} ({})'.format(speedType, events[eventId]['eventname'], eventId))

        for uid, speed in uids.items():
            if speedType not in sessionBests or uid not in sessionBests[speedType]:
                if count == 0:
                    f.write('### Ghost Rankings\n\n')
                    f.write('These rankings do not have a valid session associated with the event.\n\n')

                    f.write('| Speed Type | User Name | User ID | Speed (knots) |\n')
                    f.write('| ---------- | --------- | :-----: | :-----------: |\n')

                f.write('| {} | {} | {} | {:.2f} |\n'.format(speedTypes[speedType], users[uid], uid, speedKnots(speed)))

                addToRebuilds(rebuilds, uid, speedType)

                count += 1

    if count:
        f.write('\n')

    return count
               

def reportMissing(f, eventId, rankingBests, sessionBests, rebuilds):
    '''Report missing rankings'''

    count = 0

    for speedType, uids in sessionBests.items():
        if speedType not in rankingBests:
            print('Warning: {} not in event rankings of {} ({}))'.format(speedType, events[eventId]['eventname'], eventId))

        for uid, speed in uids.items():
            if speedType not in rankingBests or uid not in rankingBests[speedType]:
                if speed > 0:
                    if count == 0:
                        f.write('### Missing Rankings\n\n')
                        f.write('These rankings are missing but valid sessions are present.\n\n')

                        f.write('| Speed Type | User Name | User ID | Speed (knots) |\n')
                        f.write('| ---------- | --------- | :-----: | :-----------: |\n')

                    f.write('| {} | {} | {} | {:.2f} |\n'.format(speedTypes[speedType], users[uid], uid, speedKnots(speed)))

                    addToRebuilds(rebuilds, uid, speedType)

                    count += 1

    if count:
        f.write('\n')

    return count


def reportIncorrect(f, eventId, rankingBests, sessionBests, rebuilds):
    '''Report incorrect rankings'''

    count = 0

    for speedType, uids in sessionBests.items():
        if speedType in rankingBests:
            for uid, speed in uids.items():
                if uid in rankingBests[speedType]:
                    if rankingBests[speedType][uid] != speed:
                        if count == 0:
                            f.write('### Incorrect Rankings\n\n')
                            f.write('These rankings are present but do not match the fastest valid session.\n\n')

                            f.write('| Speed Type | User Name | User ID | Ranking (knots) | Session (knots) |\n')
                            f.write('| ---------- | --------- | :-----: | :-------------: | :-------------: |\n')

                        f.write('| {} | {} | {} | {:.2f} | {:.2f} |\n'.format(
                            speedTypes[speedType], users[uid], uid,
                            speedKnots(rankingBests[speedType][uid]), speedKnots(speed)))

                        addToRebuilds(rebuilds, uid, speedType)

                        count += 1

    if count:
        f.write('\n')

    return count


def checkRankings(apiName, f1, eventId, rankingBests, rankingDuplicates, sessionBests):
    """Check rankings against sessions"""

    rebuilds = {}

    filename = os.path.join(projdir, 'docs', apiName, 'events', str(eventId) + '.md')  
    dirname = os.path.dirname(filename)
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    with open(filename, "w") as f2:
        f2.write('## {} - ID {}\n\n'.format(events[eventId]['eventname'], eventId))

        startDate = datetime.strptime(events[eventId]['start_date'][:10], '%Y-%m-%d').strftime('%-d %b %Y')
        endDate = datetime.strptime(events[eventId]['end_date'][:10], '%Y-%m-%d').strftime('%-d %b %Y')
        f2.write('Dates: {} - {}\n\n'.format(startDate, endDate))

        f2.write('Links to GP3S: [Event]({}{}), [Rankings]({}{}), [Sessions]({}{})\n\n'.format(
            'https://www.gps-speedsurfing.com/default.aspx?mnu=event&val=', eventId,
            'https://www.gps-speedsurfing.com/default.aspx?mnu=eventranking&val=', eventId,
            'https://www.gps-speedsurfing.com/default.aspx?mnu=eventsessions&val=', eventId))

        duplicates = reportDuplicates(f2, eventId, rankingDuplicates, rebuilds)
        ghosts = reportGhosts(f2, eventId, rankingBests, sessionBests, rebuilds)
        missing = reportMissing(f2, eventId, rankingBests, sessionBests, rebuilds)
        incorrect = reportIncorrect(f2, eventId, rankingBests, sessionBests, rebuilds)

        f2.write('### Actions Required\n\n')

        if rebuilds:
            f2.write('{} users require a rebuild in the event rankings.\n\n'.format(len(rebuilds)))

            f2.write('| User Name | User ID | Speed Types |\n')
            f2.write('| --------- | :-----: | ----------- |\n')

            for uid in sorted(rebuilds):
                f2.write('| {} | {} | {} |\n'.format(users[uid], uid, ', '.join(sorted(rebuilds[uid]))))
        else:
            f2.write('No users require a rebuild in the event rankings.\n\n')
            
    return duplicates, ghosts, missing, incorrect, len(rebuilds)

In [9]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], '..'))
    for apiName, eventsUrl in SITES.items():
        basedir = os.path.join(projdir, 'cache', apiName, 'eventranking')
        
        domain = urllib.parse.urlparse(eventsUrl).netloc

        filename = os.path.join(projdir, 'cache', apiName, 'events.json')
        with open(filename) as f:
            events = json.load(f)
        events = {int(k): v for k, v in events.items()}

        eventIds = sorted([*events.keys()], reverse=True)

        filename = os.path.join(projdir, 'docs', apiName, 'events', 'README.md')
        dirname = os.path.dirname(filename)
        if not os.path.exists(dirname):
            os.makedirs(dirname)

        dateIssuesTotal = duplicatesTotal = ghostsTotal = missingTotal = incorrectTotal = actionsTotal = 0

        with open(filename, "w") as f:
            f.write('## GP3S Events - {}\n\n'.format(domain))

            utcnow = str(datetime.utcnow()).split('.')[0]
            f.write('Summary of issues detected in the GP3S event rankings.\n\n')
            f.write('Last refreshed {} UTC.\n\n'.format(utcnow))

            f.write('| Event Name | Event ID | Date Issues | Duplicates | Ghosts | Missing | Incorrect | Actions |\n')
            f.write('| ---------- | :------: | :---------: | :--------: | :----: | :-----: | :-------: | :-----: |\n')

            for eventId in eventIds:
                eventDir = os.path.join(basedir, str(eventId))

                rankings = loadEventRankings(eventDir)
                rankingBests, rankingDuplicates = parseEventRankings(rankings)

                try:
                    sessions = loadEventSessions(apiName, eventId)
                    sessionBests, dateIssues = parseEventSessions(sessions)

                    duplicates, ghosts, missing, incorrect, actions = \
                        checkRankings(apiName, f, eventId, rankingBests, rankingDuplicates, sessionBests)

                    dateIssuesTotal += dateIssues
                    duplicatesTotal += duplicates
                    ghostsTotal += ghosts
                    missingTotal += missing
                    incorrectTotal += incorrect
                    actionsTotal += actions

                    f.write('| [{}]({}.md) | {} | {} | {} | {} | {} | {} | {} |\n'.format(
                        events[eventId]['eventname'], eventId, eventId,
                        dateIssues, duplicates, ghosts, missing, incorrect, actions))

                except FileNotFoundError:
                    # I have intentionally cached the event rankings (but not sessions) for the British Foil Speed Challenge 2022
                    pass

            f.write('| TOTAL | - | {} | {} | {} | {} | {} | {} |\n'.format(
                dateIssuesTotal, duplicatesTotal, ghostsTotal, missingTotal, incorrectTotal, actionsTotal))

    print('All done!')

All done!
